# U.T.4 Aprendizaje supervisado (II).
# Introducción a los problemas de regresión
**Qué es la regresión**

La regresión intenta predecir el valor de una variable de salida, a partir de las variables de entrada,
dentro de una escala continua en vez de un conjunto de etiquetas (clasificación)


## Regresión lineal
Intenta utilizar elemento geométrico recto (línea, plano, hiperplano) para intentar las predicciones.
La RL se puede dividir en simple o múltiple dependiendo del número de variables de entrada.
Generalmente no es necesario que las variables estén distribuidas según la normal.

Los modelos de regresión lineal son muy dependientes (LinearRegression) de los valores extremos.

la clase RANSACRegressor utiliza un algoritmo de detección de extremos bastante potente, pero no debemos olvidar que
habrá que estudiar en profundidad el problema y los extremos para determinar qué hacer con ellos.
El parámetro residual_threshold de la clase RANSACRegressor, es dependiente del problema que se esté tratando y
no se puede determinar más que con el mecanismo de prueba y error.

Para medir el rendimiento podemos usar la métrica MSE.

Al igual que en el modelo de clasificación podemos usar las regularizaciones L2 (Lasso), L1(Ridge) o Elastic para
hacer la regresión

In [13]:
from sklearn.linear_model import LinearRegression, RANSACRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Estimación de modelo
df = pd.read_csv('housing.data.txt', header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
X = df[['RM']].values
y = df['MEDV'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


slr = LinearRegression()
slr.fit(X, y)
y_pred = slr.predict(X)
print('Slope: %.3f' % slr.coef_[0])
print('Intercept: %.3f' % slr.intercept_)

# Medimos Regresión Lineal
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (
    mean_squared_error(y_train, y_train_pred),
    mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
    r2_score(y_train, y_train_pred),
    r2_score(y_test, y_test_pred)))


Slope: 9.102
Intercept: -34.671
MSE train: 42.179, test: 46.911
R^2 train: 0.502, test: 0.437


## Regresión polinomial
Intenta utilizar un ecuación polinomial para intentar las predicciones.
La aproximación es la misma que la llevada en la clasificación con los mismos problemas de sobre ajuste,
por lo que la elección del grado del polinomio debe hacerse de forma muy cuidadosa.

In [7]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

regr = LinearRegression()

quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)
X_quad = quadratic.fit_transform(X)
X_cubic = cubic.fit_transform(X)

X_fit = np.arange(X.min(), X.max(), 1)[:, np.newaxis]

regr = regr.fit(X_quad, y)
y_quad_fit = regr.predict(quadratic.fit_transform(X_fit))
quadratic_r2 = r2_score(y, regr.predict(X_quad))

regr = regr.fit(X_cubic, y)
y_cubic_fit = regr.predict(cubic.fit_transform(X_fit))
cubic_r2 = r2_score(y, regr.predict(X_cubic))

quadratic_r2, cubic_r2

(0.5484256373971057, 0.561225722635539)

## Regresión usando bosques aleatorios
El modelo usará la misma técnica que se vio en la clasificación, ajustando un conjunto de árboles de decisión
independientes. La ventaja de este método es que no necesita ningún tipo de transformación de las características.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv('housing.data.txt', header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS',
              'NOX', 'RM', 'AGE', 'DIS', 'RAD',
              'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

forest = RandomForestRegressor(n_estimators=1000, criterion='mse', random_state=1, n_jobs=-1)

forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
    mean_squared_error(y_train, y_train_pred),
    mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
    r2_score(y_train, y_train_pred),
    r2_score(y_test, y_test_pred)))

MSE train: 6.325, test: 50.921
R^2 train: 0.921, test: 0.436


## Regresión usando SVM
También se pueden usar las máquinas de soporte vector para problemas de regresión. Con este algoritmo es muy importante
escalar las características.

In [9]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('housing.data.txt', header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS',
              'NOX', 'RM', 'AGE', 'DIS', 'RAD',
              'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

X = df.iloc[:, :-1].values
y = np.array(df['MEDV'].values).reshape(-1, 1)

sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

svm = SVR(kernel='rbf')
svm.fit(X_train, y_train.ravel())
y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
    mean_squared_error(y_train, y_train_pred),
    mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
    r2_score(y_train, y_train_pred),
    r2_score(y_test, y_test_pred)))



MSE train: 0.150, test: 0.197
R^2 train: 0.842, test: 0.816
